# Question 1
## Developing an Artificial Neural Network from Scratch.

In this notebook, we will be developing a fully connected feedforward neural network.

We will import the MNIST dataset from keras datsets. The MNIST dataset contains images of 28x28 pixels each having values ranging from 0-255.
It has 60000 images in the training set and 10000 images in the test set. However, we will only use the first 10000 images for training and first 1000 images for testing because our code isn't optimized and it takes time to run. We are not looking for accuracy of our network right now, we will be doing that in the next question when we will be implementing the same using Tensorflow.


Run the first 3 cells. Your code begins after that.

In [1]:
import numpy as np
from keras.datasets import mnist
import random

2024-08-06 19:53:23.976852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 19:53:23.976987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 19:53:24.169462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


As discussed in the class, the images are flattened to a column.

Then we are normalizing them by dividing by 255.

In [3]:
train_X=train_X.reshape(60000,784,1)    # flattening
test_X=test_X.reshape(10000,784,1)

train_y=train_y.reshape(60000,1)
test_y=test_y.reshape(10000,1)

train_X= train_X/255
test_X = test_X/255

train_X=train_X[:10000]         #taking the first 10000 images.
train_y=train_y[:10000]
test_X=test_X[:1000]
test_y=test_y[:1000]
train_data=list(zip(train_X,train_y))
test_data=list(zip(test_X,test_y))

## 1.1 Write the code for Sigmoid Function.

In [4]:

def sigmoid(z):
  t = np.exp(-z)
  return (1/(1+t))

## 1.2 The Network

We will making a class called Network which has certain functions inside it. The cost function used is Cross-Entropy Loss. You need to code only the first 3. Rest are done for you.  There are various places within the code marked as stop_zone. Read the instructions below the code at those places to check whether your code till there is correct or not.

In [5]:
class Network(object):
    def __init__(self,sizes): # sizes is a list containing the network.
                              # eg : [784,128,10] means input =784 neurons,
                              #    1st hidden layer 128 neurons, output 10 neurons.
        self.sizes=sizes
        self.num_layers=len(sizes)
        self.weights= [np.random.randn(x,y) for x,y in zip(sizes[1:],sizes[:-1])]
        self.biases= [np.random.randn(y,1) for y in sizes[1:]]
    def show(self): # Define the 'show' method within the 'Network' class
         print(self.num_layers)
         for bias in self.biases:
          print(bias.shape)
         for weight in self.weights:
          print(weight.shape)
    def forwardpropagation(self,a):
        for b,w in zip(self.biases, self.weights):
            a=sigmoid(np.dot(w,a)+b) # sig (w.a +b)
            print(a.shape)
        return a
    def backpropagation(self,x,y):


        y_t = np.zeros((len(y), 10))
        y_t[np.arange(len(y)), y] = 1
        y_t= y_t.T


        nabla_b=[np.zeros(b.shape) for b in self.biases]
        nabla_w=[np.zeros(w.shape) for w in self.weights]

        # initially, a0 = input.
        activation=x
        activation_list=[x]

        # step 1 : calculation of delta in last layer

        # write the same forward propagation code here but while doing so store the a's.
        for w,b in zip(self.weights,self.biases):
            activation=sigmoid(np.dot(w,activation)+b)
            activation_list.append(activation)

        delta= (activation_list[-1]-y_t)*activation_list[-1]*(1-activation_list[-1])


        # step 2 : nabla_b and nabla_w relation with delta of last layer

        nabla_b[-1]=delta
        nabla_w[-1]= np.dot(delta,activation_list[-2].transpose())

        print("{} {}".format(nabla_b[-1].shape,nabla_w[-1].shape) )

        return nabla_b , nabla_w

    def SGD(self, train_data,epochs,mini_batch_size, lr):
        n_train= len(train_data)
        for i in range(epochs):
            random.shuffle(train_data)
            mini_batches = []
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, lr)
            print("Epoch {} done".format(i))

        print(np.array(mini_batches, dtype=object).shape)

            # for mini_batch in mini_batches:
            #     self.update_mini_batch(mini_batch,lr)

            # self.predict(train_data)
            # print("Epoch {0} completed.".format(i+1))

    # the functions below are complete. If you are fine till stop_zone 5, you can run
    # this whole cell and train, test the data by running the last cell of the notebook.
    # You may need to wait for around 10 minutes to see the test predictions.

    def update_mini_batch(self,mini_batch,lr):
        nabla_b=[np.zeros(b.shape) for b in self.biases]
        nabla_w=[np.zeros(w.shape) for w in self.weights]
        for x,y in mini_batch:
            delta_b,delta_w= self.backpropagation(x,y)
            nabla_b=[nb+ db for nb,db in zip (nabla_b,delta_b)]
            nabla_w=[nw+dw for nw,dw in zip(nabla_w,delta_w)]

        self.weights=[w- lr*nw/len(mini_batch) for w,nw in zip(self.weights,nabla_w)]
        self.biases=[b-lr*nb/len(mini_batch) for b,nb in zip(self.biases,nabla_b)]

    def predict(self,test_data):
        test_results = [(np.argmax(self.forwardpropagation(x)),y) for x,y in test_data]
        # returns the index of that output neuron which has highest activation

        num= sum(int (x==y) for x,y in test_results)
        print ("{0}/{1} classified correctly.".format(num,len(test_data)))



In [6]:
#stop-zone 1 results
def show(self):
     print(self.num_layers)
     for bias in self.biases:
      print(bias.shape)
     for weight in self.weights:
      print(weight.shape)
net=Network([784,128,64,10])
net.show()

4
(128, 1)
(64, 1)
(10, 1)
(128, 784)
(64, 128)
(10, 64)


In [7]:
#stop zone 2 results
net=Network([784,128,64,10])
input_data = train_X[0]
print(train_X[0].shape) # Print the shape of the input
net.forwardpropagation(input_data)

(784, 1)
(128, 1)
(64, 1)
(10, 1)


array([[0.99998521],
       [0.17446791],
       [0.99664891],
       [0.0723137 ],
       [0.99853323],
       [0.99956076],
       [0.00219064],
       [0.01141615],
       [0.13977496],
       [0.98215848]])

In [8]:
#stop zone 3 results
net=Network([784,128,64,10])
net.backpropagation(train_X[0],train_y[0])

(10, 1) (10, 64)


([array([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [

In [9]:
#stop zone 4 results
net=Network([784,128,64,10])
nabla_b,nabla_w=net.backpropagation(train_X[0],train_y[0])
for nb in nabla_b:
  print(nb.shape)
for nw in nabla_w:
  print(nw.shape)

(10, 1) (10, 64)
(128, 1)
(64, 1)
(10, 1)
(128, 784)
(64, 128)
(10, 64)


In [10]:
#stop zone 5 results
net=Network([784,256,128,64,10])
net.SGD(train_data=train_data,epochs=20,mini_batch_size=20,lr=0.01)

Epoch 0 done
Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done
Epoch 6 done
Epoch 7 done
Epoch 8 done
Epoch 9 done
Epoch 10 done
Epoch 11 done
Epoch 12 done
Epoch 13 done
Epoch 14 done
Epoch 15 done
Epoch 16 done
Epoch 17 done
Epoch 18 done
Epoch 19 done
(0,)


In [11]:
net=Network([784,128,64,10])
net.SGD(train_data=train_data,epochs=10,mini_batch_size=20,lr=0.01)
print("Test data:")
net.predict(test_data)

Epoch 0 done
Epoch 1 done
Epoch 2 done
Epoch 3 done
Epoch 4 done
Epoch 5 done
Epoch 6 done
Epoch 7 done
Epoch 8 done
Epoch 9 done
(0,)
Test data:
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128, 1)
(64, 1)
(10, 1)
(128

/tmp/ipykernel_33/4165008963.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  num= sum(int (x==y) for x,y in test_results)


# End of question 1